In [45]:
import os
import sys
import re
import pandas as pd
import json
from collections import Counter
import matplotlib.pyplot as plt

In [77]:
news = open("seed_channel_list/usa_news_channel_list.json", 'r', encoding="utf-8")
df = pd.read_json(news)
channels, urls = [], []
for channel, url in df["channels"]:
    channels.append(channel.replace(" ",""))
    urls.append(url)
    
print(len(channels))
len(set(urls))


55


55

In [78]:
pattern = r"(?<=output_).[^_]*(?=_)"
dir_name = "outputs_usa_news"
output_file_name = "video_list/usa_news_Apr012024.csv"
channel_list = set()
files = [f"{dir_name}/{f}" for f in os.listdir(f"{dir_name}/") if f.endswith(".csv")]

for f in files:
    channel_name = re.search(pattern, f)[0]
    channel_list.add(channel_name)
    print(f)


outputs_usa_news/output_8NewsNowVegas_0.csv
outputs_usa_news/output_8NewsNowVegas_1.csv
outputs_usa_news/output_8NewsNowVegas_2.csv
outputs_usa_news/output_8NewsNowVegas_3.csv
outputs_usa_news/output_ActionNewsNow_0.csv
outputs_usa_news/output_ActionNewsNow_2.csv
outputs_usa_news/output_ActionNewsNow_3.csv
outputs_usa_news/output_AGDAILY_0.csv
outputs_usa_news/output_AGDAILY_1.csv
outputs_usa_news/output_AGDAILY_2.csv
outputs_usa_news/output_AGDAILY_3.csv
outputs_usa_news/output_Algemeiner_0.csv
outputs_usa_news/output_Algemeiner_1.csv
outputs_usa_news/output_Algemeiner_2.csv
outputs_usa_news/output_Algemeiner_3.csv
outputs_usa_news/output_ArkTimes_0.csv
outputs_usa_news/output_ArkTimes_2.csv
outputs_usa_news/output_AustinAmerican-Statesman_0.csv
outputs_usa_news/output_AustinAmerican-Statesman_2.csv
outputs_usa_news/output_AustinAmerican-Statesman_3.csv
outputs_usa_news/output_CasperStarTribune_0.csv
outputs_usa_news/output_CasperStarTribune_2.csv
outputs_usa_news/output_cbs21_0.csv
o

In [79]:
all_df_list = []
for channel in channel_list:
    channel_files = [f for f in files if f.startswith(f"{dir_name}/output_{channel}")]
    channel_df_list = []

    for f in channel_files:
        df_temp = pd.read_csv(f, encoding= 'unicode_escape')
        channel_df_list.append(df_temp)

    df = pd.concat(channel_df_list).drop_duplicates(subset=["video_id"], keep='first').reset_index(drop=True)
    all_df_list.append(df)
    
df = pd.DataFrame()
df = pd.concat(all_df_list).drop_duplicates(subset=["video_id"], keep='first').reset_index(drop=True)
df

,video_id,channel_name,video_title,view_count
0,aO4JW1DHVoU,KVAL,New Approach for Arcimoto After Years of Overv...,6100
1,nO4qot1A2SM,KVAL,Adderall Shortage Creating Nightmare Scenario ...,2000
2,n4vWQpYL1hc,KVAL,Fiesta Bowl Media Day,1200
3,RGHEZA7k3Oc,KVAL,Brave Bystander Knocks Out Knife-Wielding Carj...,728
4,yOa_D0gC5C4,KVAL,Former Eugene Police Officer Says Department i...,600
...,...,...,...,...
17626,0tSIJuoX0m0,KXIITV,Caddo resident speaks out about downed stop si...,45
17627,es3c4xE7o7A,KXIITV,Ardmore city commissioners approve zoning chan...,44
17628,n3-5Tt1cZds,KXIITV,Oklahoma schools enter partnership with contro...,42
17629,D4ksN9n7Bqw,KXIITV,Suspect in Sherman shooting on the run,42


In [80]:
df = df[df['view_count'] > 10000].reset_index(drop=True)
df = df.sample(frac=1, random_state=1).reset_index(drop=True)
df

,video_id,channel_name,video_title,view_count
0,RM7Y9NTDfiY,KOMONews,"Amid crime surge in Seattle, some take steps t...",199000
1,B4LkGl0qq-A,fox26houston,Laser strikes hitting airplanes at record pace,64000
2,-A-4-mcgU_Q,8NewsNowVegas,"Metro officers involved in car chase, shooting...",32000
3,DzkoyiHLRI8,FOX35Orlando,New social media trends leaves people terroriz...,301000
4,-im-Xjw4MAI,ClickOnDetroit,Here's why the Detroit Lions wear Honolulu Blue,120000
...,...,...,...,...
5578,v9fOMEGuEyg,CourtTV,New Details in Bryan Kohbergerâs Arrest & Ex...,230000
5579,pSY6eygx6hQ,WUSA9news,70% of Americans don't think Joe Biden should ...,21000
5580,t7jVKpntgNw,KOMONews,"Seattle vandalized as arrests made, and people...",27000
5581,GwsMDxDPHiQ,fox26houston,"Woman linked to million dollar credit scheme, ...",58000


In [81]:
Counter(df["channel_name"])

Counter({'WUSA9news': 561,
         'kcalnews': 549,
         'FOX35Orlando': 504,
         'CourtTV': 459,
         'CBSNews': 374,
         '8NewsNowVegas': 368,
         'Fox10Phoenix': 364,
         'fox26houston': 348,
         'Denver7': 317,
         'NBC10Philadelphia': 212,
         'ClickOnDetroit': 210,
         'WBIR10': 195,
         'KGUN9': 153,
         'HawaiiNewsNow': 148,
         'KOMONews': 124,
         'MotleyFool': 117,
         'harvardbusinessreview': 99,
         'AustinAmerican-Statesman': 88,
         'wpdeabc15': 63,
         'KOB': 51,
         'KiviTVNews': 47,
         'MarketBeat': 47,
         'USCourts': 30,
         'WNEMTV5': 26,
         'VerifyThis': 16,
         'TimesandDemocrat': 15,
         'IowasNewsNow': 14,
         'KCRG': 11,
         'WHIO7': 9,
         'TheDailyRecord': 7,
         'YorkDailyRecord': 6,
         'ActionNewsNow': 6,
         'ArkTimes': 5,
         'GazetteTimes': 4,
         'TheMarshallProject': 4,
         'KTTC': 

In [82]:
df.to_csv(output_file_name, index=False, encoding="utf-8")